In [ ]:
using ESDL
using ESDLPlots
using ProgressMeter
gr()

In this Notebook we want to visualize the regions where cube variable take on their highest or lowest value. To do this we calculate the 99% and the 1% quantile for each variable from a subset of the data and afterwards count how often the quantile was exceeded for each variable in each grid cell.

In [ ]:
c             = Cube()
vars          = ["air_temperature_2m","soil_moisture","gross_primary_productivity"];
cdata         = getCubeData(c,region="South America",variable=vars);

In [ ]:
import OnlineStats: Hist
d = mapCube(Hist,cdata,100,by=(VariableAxis,))

In [ ]:
plotHist(d)

In [ ]:
qu = quantile(d,[0.01,0.99])

In [ ]:
plotXY(qu)

Get the quantiles for the subsample

Define a function that counts quantile crossings for each grid cell

In [ ]:
"""
How often the upper or lower quantiles are crossed in each time series
"""
function countExtremes(xout::AbstractArray,xin::AbstractVector,qvec::AbstractVector)
    nlow,nhigh=0,0
    qlow,qhigh=qvec
    for v in xin
        if !isnan(v)
            v<=qlow && (nlow+=1)
            v>=qhigh && (nhigh+=1)
        end
    end
    xout[1]=nlow
    xout[2]=nhigh
end

In [ ]:
import ESDL.NaNMissing
indims = InDims("Time",miss=NaNMissing()),InDims("Quantile",miss=NaNMissing())
outdims = OutDims(CategoricalAxis("Direction",["High","Low"]),outtype=Int,miss=NaNMissing())
o=mapCube(countExtremes,(cdata,qu),
  indims = indims,
  outdims = outdims,no_ocean=1)

Now we plot the low extremes

In [ ]:
for i=1:3
display(o.axes[4].values[i])
display(plotMAP(o,dmax=10,dir=1,var=i))
end

And the high extremes

In [ ]:
for i=1:3
display(o.axes[4].values[i])
display(plotMAP(o,dmax=10,dir=2,var=i))
end